# ProjF5 - Final Update [20%]. 
### You will provide details about your model, how you did the training and model selection, and its performance compared to the baseline.


In [1]:
from datasets import load_dataset, load_metric
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
from transformers import pipeline, set_seed, AutoModelForSeq2SeqLM, AutoTokenizer

from tqdm import tqdm
import torch

In [2]:
dataset = load_dataset("cnn_dailymail", "3.0.0")

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [4]:
model_name = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

Split Lists into Chunks (useful to make batch-sized chunks)

In [5]:
def chunks(list_ele, batch_size):
  for i in range(0, len(list_ele), batch_size):
    yield list_ele[i : i+batch_size]

In [ ]:
def evaluate_pegasus(dataset, metric, model, tokenizer, batch_size=16, device='cpu', column_text="article", column_summary="highlights"):
    article_batches = [dataset[column_text][i:i+batch_size] for i in range(0, len(dataset[column_text]), batch_size)]
    target_batches = [dataset[column_summary][i:i+batch_size] for i in range(0, len(dataset[column_summary]), batch_size)]

    for article_batch, target_batch in tqdm(zip(article_batches, target_batches), total=len(article_batches)):
        # tokenize the articles
        inputs = tokenizer(article_batch, max_length=1024, truncation=True, padding="max_length", return_tensors="pt")

        #generate the summaries
        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                                   attention_mask=inputs["attention_mask"].to(device),
                                   max_length=128,
                                   num_beams=8,
                                   length_penalty=1.1,
                                   early_stopping=True)

        decoded_summaries = [tokenizer.decode(summary, skip_special_tokens=True, clean_up_tokenization_spaces=True) for summary in summaries]
        # update the metric
        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    score = metric.compute()
    return score